In [1]:
##########################################
####        Importing Pachages        ####
##########################################

from __future__ import division, print_function
import kf_book.pf_internal as pf_internal
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import uniform
from filterpy.stats import plot_gaussian_pdf
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import operator
from collections import Counter
import pickle
import math as m
import sys
import scipy
from scipy.spatial import distance
import numpy.random
from matplotlib.patches import Circle
from numpy.random import uniform
from operator import itemgetter 

import scipy.stats
import time
import paho.mqtt.client as mqtt
import threading
import os
import re

global particles
global weight
global N
global sensor_std_err
global plot_particles
global xlim
global ylim
global xbins
global ybins
global xs
global motion_sensors
global activity_sensors
global beacons
global beacons_mean
global beacons_stds
global fig
global ax
global oracle

oracle = []
estimation = []
plot_particles = False
motion_sensors = []
activity_sensors = []
sensor_std_err=.1
# Information Granularity
realx = 6.6
realy = 10.6
gridsize = 1
EPSILON = sys.float_info.epsilon  # smallest possible difference
xbins = int(np.ceil(realx) / gridsize)
ybins = int(np.ceil(realy) / gridsize)
xlim=(0, xbins)
ylim=(0, ybins)


In [2]:
##########################################
####  PF Internal Methods Definitions ####
##########################################

def simple_resample(particles, weights):
    N = len(particles)
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1. # avoid round-off error
    indexes = np.searchsorted(cumulative_sum, random(N))

    # resample according to indexes
    particles[:] = particles[indexes]
    weights.fill(1.0 / N)
    
def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights.fill(1.0 / len(weights))    

def neff(weights):
    return 1. / np.sum(np.square(weights))

def create_uniform_particles(x_range, y_range, hdg_range, N):
    particles = np.empty((N, 3))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N)
    particles[:, 2] = uniform(hdg_range[0], hdg_range[1], size=N)
    particles[:, 2] %= 2 * np.pi
    return particles

def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 3))
    particles[:, 0] = mean[0] + (randn(N) * std[0])
    particles[:, 1] = mean[1] + (randn(N) * std[1])
    particles[:, 2] = mean[2] + (randn(N) * std[2])
    particles[:, 2] %= 2 * np.pi
    return particles


def DistanceToRSSI(agent_pos):
    rssi_values = []
    for b in beacons:
        dist = int(round(np.linalg.norm(b - agent_pos)))
        
        # Max reading is (7 / gridsize) meters
        if (dist > int(7 / gridsize)):
            dist = int(7 / gridsize)
        
        # print(dist)
        mean = beacons_mean[dist - 1]
        std = beacons_stds[dist - 1]
        
        sample = np.random.normal(mean, std, 1)
        rssi_values.append(sample[0])
        
    return rssi_values

def interpolate(inp, fi):
    i = int(fi)
    f = fi - i
    return (inp[i] if f < EPSILON else
            inp[i] + f*(inp[i+1]-inp[i]))

#------------------------------------
#----------------------- Update Step:

def SensorModel(distances):
    m = itemgetter(*distances)(beacons_mean)
    s = itemgetter(*distances)(beacons_stds)
    return [m, s]

def update(particles, weights, z, R):
    for i, beacon in enumerate(beacons):
        distances = np.round((np.linalg.norm(particles[:, 0:2] - beacon, axis=1))/10).astype(int)
        ms, ss = SensorModel(distances)
        # weighting function
        weights += np.abs(np.log10(scipy.stats.norm(ms, ss).pdf(z[i])))
        
    weights += 1.e-300      # avoid round-off to zero
    
#------------------------------------
#------------- State Estimation Step:  
    
def estimate(particles, weights):
    """returns mean and variance of the weighted particles"""
    pos = particles[:, 0:2]
    mean = np.average(pos, weights=weights, axis=0)
    var  = np.average((pos - mean)**2, weights=weights, axis=0)
    return mean, var    

#------------------------------------
#------------- Particle Predict Step:  

def predict(particles, xbins, ybins):
    next_states = NextStates(TM, particles, xbins, ybins)
    particles[:, 0] = next_states[:, 0]
    particles[:, 1] = next_states[:, 1]
    return particles

#------------------------------------
#----------------- Transition Matrix: 

def transition_matrix(transitions, xbins, ybins):
    M = np.ones((xbins * ybins, xbins * ybins))
    for (state, state_p) in zip(transitions,transitions[1:]):
        i = xbins * state[1] + state[0]
        j = xbins * state_p[1] + state_p[0]
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def CumulativeSum(m):
    return m.cumsum()

def returnobj(cdf):
    randnum = np.random.random()
    for i in range(len(cdf) - 1):
        if((randnum>cdf[i]) and (randnum<=cdf[i + 1])):
            return i

def IndexToState(index, xbins, ybins):
    y = int(np.floor(index / xbins))
    x = index - xbins * y
    return x, y

def StateToIndex(x, y, xbins, ybins):
    i = xbins * np.floor(y) + np.floor(x)
    return int(i)
        
def NextStates(TM, particles, xbins, ybins):
    cdf = np.apply_along_axis(CumulativeSum, 1, TM)
    zero = np.array([0]) # np.zeros((len(cdf[:,1]))
    c = np.tile(zero[np.newaxis,:], (cdf.shape[0],1))
    cdf = np.concatenate((c, cdf), axis=1)
    next_states = np.empty((len(particles), 2))
    
    for i in range(len(particles)):
        current_state_x = particles[i, 0]
        current_state_y = particles[i, 1]
        
        row = StateToIndex(current_state_x, current_state_y, xbins, ybins)
        next_state_index = returnobj(cdf[row])
        next_states[i, 0] = IndexToState(next_state_index, xbins, ybins)[0]
        next_states[i, 1] = IndexToState(next_state_index, xbins, ybins)[1]

    return next_states

def beacon_sensor_model():
    filename = '../RSSI.csv'

agenttrace = 'AgentTrace.csv'
f = open(agenttrace)
temp_oracle_raw = []
gridized_data = []


with open(agenttrace, 'r') as user_data:
    reader = csv.reader(user_data, delimiter=',', quotechar=',')
    for row in reader:
        if (reader.line_num > 1):
            temp_oracle_raw.append([float(row[2]), float(row[3]), str(row[5])])

temp_oracle = []
for i in range(len(temp_oracle_raw)):
     temp_oracle.append([temp_oracle_raw[i][0] / gridsize, temp_oracle_raw[i][1] / gridsize , temp_oracle_raw[i][2]])
            
for d in temp_oracle:
    gridized_data.append([int(np.floor(d[0])), int(np.floor(d[1]))])        

beacons = np.array([[1, 1], [3, 4.9], [1.35, 7.7], [3.3, 10.5], [3.3, 10.5], [6, 0.5], [0.5, 3.3], [6, 10.5]])    

for i in range(len(beacons)):
    beacons[i][0] = beacons[i][0]/gridsize
    beacons[i][1] = beacons[i][1]/gridsize

TM = transition_matrix(gridized_data, xbins, ybins)    


In [3]:
##########################################
####     Sensors class Definitions    ####
##########################################

class PressureSensor:
    radius = float()
    x = float()
    y = float()
    attached_object = str()
    active = bool()
    
    def Initialize(self, x, y, r, obj):
        self.x = x
        self.y = y
        self.radius = r
        self.attached_object = obj
        self.active = False
        
    def Fire(self):
        self.active = True
        
    def Deactive(self):
        self.active = False

class MotionSensor:
    x = float()
    y = float()
    r = float()
    active = bool()
    
    def Initialize(self, x, y, r):
        self.x = x
        self.y = y
        self.r = r
        self.active = False
        
    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
        
class WaterFlowSensor:
    x = float()
    y = float()
    attached_object = str()
    active = bool()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj

    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
    
class VoltageSensor:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj
    
    def Fire(self):
        self.active = True

    def Deactive(self):
        self.active = False
        
class ElectricitySensor:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj
    
    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
    
class Accelerometer:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.attached_object = obj
        self.active = False
        
    def Fire(self):
        self.active = True
        
    def Deactive(self):
        self.active = False

In [4]:
##########################################
####  Space and Sensor Model Config.  ####
##########################################


NL = len(beacons)

pik = 'SensorModel.dat'

with open(pik, 'rb') as f:
    beacons_mean = pickle.load(f)
    beacons_stds = pickle.load(f)
    
new_len = int(len(beacons_mean) / gridsize)
delta = (len(beacons_mean)-1) / float(new_len-1)
new_beacons_mean = [interpolate(beacons_mean, i*delta) for i in range(new_len)]

new_len = int(len(beacons_stds) / gridsize)
delta = (len(beacons_stds)-1) / float(new_len-1)
new_beacons_stds = [interpolate(beacons_stds, i*delta) for i in range(new_len)]

beacons_mean = new_beacons_mean
beacons_stds = new_beacons_stds

sensordefs = 'BIMSensorDef.csv'
f = open(sensordefs)
oracle_raw = []
gridized_data = []

with open(sensordefs, 'r') as sensor:
    reader = csv.reader(sensor, delimiter=',', quotechar=',')
    for row in reader:
        # print(row)
        if(row[0] == 'pressure'):
            pressure_sensor = PressureSensor()
            pressure_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, 2 / gridsize, row[1])
            activity_sensors.append(pressure_sensor)
        
        elif(row[0] == 'water'):
            water_sensor = WaterFlowSensor()
            water_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(water_sensor)
        
        elif(row[0] == 'electricity'):
            voltage_sensor = VoltageSensor()
            voltage_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(voltage_sensor)
        
        elif(row[0] == 'accelerometer'):
            acc = Accelerometer()
            acc.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(acc)


k = 0
for i in range(xbins + 1):
    for j in range(ybins + 1):
        ms = MotionSensor()
        ms.Initialize(i, j, 1)
        motion_sensors.append(ms)
        k = k + 1

In [5]:
##########################################
####   Activity Recognition Methods   ####
##########################################            
def ActivitiesSensorGeneration(data):
    loc_x = data[0]
    loc_y = data[1]
    
    
    for motion in motion_sensors:
        circ = Circle((motion.x, motion.y), motion.r)
        if (circ.contains_point([loc_x, loc_y])):
            motion.Fire()
    
    
    for sensor in activity_sensors:
        activity = data[2].lower()

        if (activity == 'sleep' or activity == 'sit' or activity == 'eat'):
            if(type(sensor) == PressureSensor):
                if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 2 / gridsize):
                    sensor.Fire()

        
        elif (activity == 'wash' or activity == 'clean'):
            if(type(sensor) == WaterFlowSensor):
                if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 2 / gridsize):
                    sensor.Fire()

        elif (activity == 'exercise'):
            if(type(sensor) == Accelerometer):
                if(sensor.attached_object == 'Agent'):
                    sensor.Fire()
        
        elif (activity == 'exercise' or activity == 'cook'):
            if(type(sensor) == Accelerometer):         
                if(sensor.attached_object == 'Refrigerator-Side-by-Side-with_Ice_and_Water_Dispenser'):
                    if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 1.3 / gridsize):
                        sensor.Fire()
                        
            elif(type(sensor) == ElectricitySensor):
                if(sensor.attached_object == "Oven Tower Unit" or sensor.attached_object == "Counter Top w Sink" 
                   or sensor.attached_object == "Counter Top - Corner"):
                    if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 1 / gridsize):
                        sensor.Fire()
                    
        elif (activity == 'entertainment'):
            if(type(sensor) == ElectricitySensor):
                if(sensor.attached_object == 'TV - Flat Screen'):
                    sensor.Fire()

def ResetSensorStates():
    for sensor in activity_sensors:
        sensor.Deactive()
        
    for motion in motion_sensors:
        motion.Deactive()
        
def RecognizeActivity(current_location, previous_location, particles):
    for motion in motion_sensors:
        if (motion.active):
            particles[:,0] = motion.x
            particles[:,1] = motion.y
    
    for sensor in activity_sensors:
        if (sensor.active):
            if (type(sensor) == Accelerometer):
                if (sensor.attached_object == 'Refrigerator-Side-by-Side-with_Ice_and_Water_Dispenser'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'cook'
                
                elif (sensor.attached_object == 'Agent'):
                    return 'exercise'
            
            elif (type(sensor) == ElectricitySensor):
                if (sensor.attached_object == 'Oven Tower Unit' or sensor.attached_object == 'Counter Top w Sink' 
                    or sensor.attached_object == 'Counter Top - Corner'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'cook'
                
                if (sensor.attached_object == 'TV - Flat Screen'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'entertainment'
                
            elif (type(sensor) == WaterFlowSensor):
                if (sensor.attached_object == 'Sink Kitchen-Single'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'clean'
                
            elif (type(sensor) == PressureSensor):
                if (sensor.attached_object == 'Chair-Breuer' or sensor.attached_object == 'Dining Table w Chairs'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'sit'
                
                elif (sensor.attached_object == 'Chair-Corbu' or sensor.attached_object == 'Sofa-Corbu'
                      or sensor.attached_object == 'bed'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'sleep'
                
    if (distance.euclidean(current_location, previous_location) <= 0.05):
        return 'nothing'
        
    else:
        return 'walk'

In [6]:
##########################################
####     Particle Filter Main Body    ####
##########################################      
global test1

def run_pf1(data):   
    agent_trace = data
    oracle.append(agent_trace)
    agent_pos = [agent_trace[0], agent_trace[1]]
    
    particles = create_uniform_particles((0,xbins), (0,ybins), (0, 1.28), N)    
    weights = np.ones(N) / N
    
    ActivitiesSensorGeneration(agent_trace)
    particles = predict(particles, xbins, ybins)        

    try:
        activity = RecognizeActivity(xs[len(xs) - 1], xs[len(xs) - 2], particles)
    except:
        activity = RecognizeActivity(0, 0, particles)

    zs =  DistanceToRSSI(agent_pos)
    update(particles, weights, z = zs, R = sensor_std_err)
    mu, var = estimate(particles, weights)
    xs.append(mu)

    # print(agent_trace[2] + ' VS ' + RecognizeActivity(xs[len(xs) - 1], xs[len(xs) - 2]))
        
    # if plot_particles:
        # plt.scatter(particles[:, 0], particles[:, 1], color='k', marker=',', s=1)

    # p1 = plt.scatter(agent_pos[0], agent_pos[1], marker='+', color='k', s=180, lw=3)
    # p2 = plt.scatter(mu[0], mu[1], marker='s', color='r')
    p1 = ax.scatter(agent_pos[0], agent_pos[1], marker='+', color='k', s=180, lw=3)
    p2 = ax.scatter(mu[0], mu[1], marker='s', color='r')

    xx = []
    yy = []
    for mo in motion_sensors:
        xx.append(mo.x)
        yy.append(mo.y)

    if plot_particles:
        alpha = .2
        if N > 5000:
            alpha *= np.sqrt(5000)/np.sqrt(N)           
        # plt.scatter(particles[:, 0], particles[:, 1], alpha=alpha, color='b')
        # plt.scatter(xx, yy, alpha=alpha, color='g')

    xs_new = np.array(xs)
    # plt.plot(xs[:, 0], xs[:, 1])
    plt.legend([p1, p2], ['Synthetic Ground Truth', 'Particle Filter Estimation'], loc=1, numpoints=1)
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    # plt.xlabel = "Synthetic Ground Truth Activity: " str(agent_trace[2]) + " Estimated Activity: " + str(RecognizeActivity(xs[len(xs) - 1], xs[len(xs) - 2]))
    fig.suptitle("Synthetic Ground Truth Activity: " + 
                 str(agent_trace[2]) + 
                 " |  Estimated Activity: " + 
                 str(activity))
    print('final position error, variance:\n\t', mu - np.array([iters, iters]), var)
    plt.show()
    fig.canvas.draw()
    ax.cla()

    ResetSensorStates()
    
    # return dd[0], mu[0], mu[1], dd[4], activity
    return "done"
    

from numpy.random import seed
N = 100
seed(2)


# %matplotlib notebook
# fig, ax = plt.figure(figsize = (7, 11))
# fig, ax = plt.subplots(1,1)

In [7]:
%matplotlib notebook
fig, ax = plt.figure(figsize = (7, 11))
fig, ax = plt.subplots(1,1)

<IPython.core.display.Javascript object>

TypeError: 'Figure' object is not iterable

In [ ]:
agenttrace = 'AgentTrace.csv'

gridsize = 1
oracle_raw = []
raw_data = []

with open(agenttrace, 'r') as user_data:
    reader = csv.reader(user_data, delimiter=',', quotechar=',')
    for row in reader:
        if (reader.line_num > 1):
            raw_data.append(row)
            oracle_raw.append([float(row[2]), float(row[3]), str(row[5])])

oracle = []
for i in range(len(oracle_raw)):
     oracle.append([oracle_raw[i][0] / gridsize, oracle_raw[i][1] / gridsize , oracle_raw[i][2]])
        
for gt in raw_data:
    time.sleep(0.6)
    run_pf1(gt)

In [ ]:
from __future__ import division, print_function
import kf_book.pf_internal as pf_internal
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import uniform
from filterpy.stats import plot_gaussian_pdf
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import operator
from collections import Counter
import pickle
import math as m
import sys
import scipy
from scipy.spatial import distance
import numpy.random
from matplotlib.patches import Circle

In [ ]:
from numpy.random import uniform

def create_uniform_particles(x_range, y_range, hdg_range, N):
    particles = np.empty((N, 3))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N)
    particles[:, 2] = uniform(hdg_range[0], hdg_range[1], size=N)
    particles[:, 2] %= 2 * np.pi
    return particles

def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 3))
    particles[:, 0] = mean[0] + (randn(N) * std[0])
    particles[:, 1] = mean[1] + (randn(N) * std[1])
    particles[:, 2] = mean[2] + (randn(N) * std[2])
    particles[:, 2] %= 2 * np.pi
    return particles

In [ ]:
class PressureSensor:
    radius = float()
    x = float()
    y = float()
    attached_object = str()
    active = bool()
    
    def Initialize(self, x, y, r, obj):
        self.x = x
        self.y = y
        self.radius = r
        self.attached_object = obj
        self.active = False
        
    def Fire(self):
        self.active = True
        
    def Deactive(self):
        self.active = False

class MotionSensor:
    x = float()
    y = float()
    r = float()
    active = bool()
    
    def Initialize(self, x, y, r):
        self.x = x
        self.y = y
        self.r = r
        self.active = False
        
    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
        
class WaterFlowSensor:
    x = float()
    y = float()
    attached_object = str()
    active = bool()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj

    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
    
class VoltageSensor:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj
    
    def Fire(self):
        self.active = True

    def Deactive(self):
        self.active = False
        
class ElectricitySensor:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj
    
    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
    
class Accelerometer:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.attached_object = obj
        self.active = False
        
    def Fire(self):
        self.active = True
        
    def Deactive(self):
        self.active = False

In [ ]:
########################
#### Configurations ####
########################

gridsize = 1
EPSILON = sys.float_info.epsilon  # smallest possible difference

# Information Granularity
realx = 6.6
realy = 10.6

xbins = int(np.ceil(realx) / gridsize)
ybins = int(np.ceil(realy) / gridsize)

def interpolate(inp, fi):
    i = int(fi)
    f = fi - i
    return (inp[i] if f < EPSILON else
            inp[i] + f*(inp[i+1]-inp[i]))


beacons = np.array([[1, 1], [3, 4.9], [1.35, 7.7], [3.3, 10.5], [3.3, 10.5], [6, 0.5], [0.5, 3.3], [6, 10.5]])
activity_sensors = []
NL = len(beacons)

pik = 'SensorModel.dat'

with open(pik, 'rb') as f:
    beacons_mean = pickle.load(f)
    beacons_stds = pickle.load(f)
    
new_len = int(len(beacons_mean) / gridsize)
delta = (len(beacons_mean)-1) / float(new_len-1)
new_beacons_mean = [interpolate(beacons_mean, i*delta) for i in range(new_len)]

new_len = int(len(beacons_stds) / gridsize)
delta = (len(beacons_stds)-1) / float(new_len-1)
new_beacons_stds = [interpolate(beacons_stds, i*delta) for i in range(new_len)]

beacons_mean = new_beacons_mean
beacons_stds = new_beacons_stds

sensordefs = 'BIMSensorDef.csv'
f = open(sensordefs)
oracle_raw = []
gridized_data = []

with open(sensordefs, 'r') as sensor:
    reader = csv.reader(sensor, delimiter=',', quotechar=',')
    for row in reader:
        # print(row)
        if(row[0] == 'pressure'):
            pressure_sensor = PressureSensor()
            pressure_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, 2 / gridsize, row[1])
            activity_sensors.append(pressure_sensor)
        
        elif(row[0] == 'water'):
            water_sensor = WaterFlowSensor()
            water_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(water_sensor)
        
        elif(row[0] == 'electricity'):
            voltage_sensor = VoltageSensor()
            voltage_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(voltage_sensor)
        
        elif(row[0] == 'accelerometer'):
            acc = Accelerometer()
            acc.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(acc)


motion_sensors = []
k = 0
for i in range(xbins + 1):
    for j in range(ybins + 1):
        ms = MotionSensor()
        ms.Initialize(i, j, 1)
        motion_sensors.append(ms)
        k = k + 1

In [ ]:
xbins

In [ ]:
#######################
##### Update Step #####
#######################

from operator import itemgetter 


def SensorModel(distances):
    m = itemgetter(*distances)(beacons_mean)
    s = itemgetter(*distances)(beacons_stds)
    
    return [m, s]

def update(particles, weights, z, R):
    # print("basdasdlkjsdhflkajshdflkajshdflkjashdflkjahsdflkjheaconsasdasd.akjshnf;klajshf;alkjsd;lkasjd;lkasjd")
    # print("and then")
    # print(particles[:, 0:2])
    # print("and then")
    # print(particles[:, 0:2]/10)
    # print("and then")
    # print(np.max(particles[:, 0:2]))
    # print("and then")
    # print(np.max(particles[:, 0:2]/10))
    for i, beacon in enumerate(beacons):
        distances = np.round((np.linalg.norm(particles[:, 0:2] - beacon, axis=1))/10).astype(int)
        # print(distances)
        ms, ss = SensorModel(distances)
        # print(ms)
        # print(ss)
        
        # weighting function
        weights += np.power(np.abs(np.log10(scipy.stats.norm(ms, ss).pdf(z[i]))), 3)
        # weights += np.abs(np.log10(scipy.stats.norm(ms, ss).pdf(z[i])))
        # print("each weight is:")
        # print(scipy.stats.norm(ms, ss).pdf(z[i]))
        # weights *= scipy.stats.norm(ms, ss).pdf(z[i])

    # print("total W for each P is:")
    # print(weights)
    weights += 1.e-300      # avoid round-off to zero
    # weights /= sum(weights) # normalize
    # print("max W is : " + str(np.max(weights)))
    # print("min W is : " + str(np.min(weights)))

In [ ]:
np.exp(2)

In [ ]:
np.searchsorted([10,20,30,40,50], 60)

In [ ]:
#################################
##### State Estimation Step #####
#################################

def estimate(particles, weights):
    """returns mean and variance of the weighted particles"""

    pos = particles[:, 0:2]
    mean = np.average(pos, weights=weights, axis=0)
    var  = np.average((pos - mean)**2, weights=weights, axis=0)
    return mean, var

In [ ]:
#############################
#### Particle Resampling ####
#############################

def simple_resample(particles, weights):
    N = len(particles)
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1. # avoid round-off error
    indexes = np.searchsorted(cumulative_sum, N)

    # resample according to indexes
    particles[:] = particles[indexes]
    weights.fill(1.0 / N)
    
def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights.fill(1.0 / len(weights))    

def neff(weights):
    return 1. / np.sum(np.square(weights))

In [ ]:
np.searchsorted([100,120,11,44,5], 150)

In [ ]:
def transition_matrix(transitions, xbins, ybins):
    # states_lookup_table = []
    # for x in range(xbins):
        # for y in range(ybins):
            # states_lookup_table.append("[" + str(x) + ", " + str(y) + "]")

    # n = 1 + len(np.array(list(set(tuple(p) for p in transitions)))) #number of unique states
    M = np.ones((xbins * ybins, xbins * ybins))
    # print("athar")
    for (state, state_p) in zip(transitions,transitions[1:]):
        # print("shadan")
        # print("state is: ")
        # print(state)
        # print("state prime is: ")
        # print(state_p)
        # i = state[0] * (ybins - 1) + state[1]
        # j = state_p[0] * (ybins - 1) + state_p[1]
        
        i = xbins * state[1] + state[0]
        j = xbins * state_p[1] + state_p[0]
        
        # print(i)
        # print(j)
        # i = state[0]*xbins + state[1]
        # j = state_p[0]*xbins + state_p[1]
        # i = states_lookup_table.index(str(state))
        # j = states_lookup_table.index(str(state_p))
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def beacon_sensor_model():
    filename = '../RSSI.csv'

In [ ]:
def transition_matrix2(transitions, xbins, ybins):
    M = np.zeros((xbins * ybins, xbins * ybins))
    
    # np.fill_diagonal(M[1:], 1)
    np.fill_diagonal(M, 1)
    # np.fill_diagonal(M[:,1:], 1)
    
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def beacon_sensor_model():
    filename = '../RSSI.csv'

In [ ]:
##################################################
####       Farnoosh and YiJi Simulation       ####
##################################################

agenttrace = 'AgentTrace.csv'
f = open(agenttrace)
#print(f.readlines)
oracle_raw = []
gridized_data = []

# Information Granularity
realx = 6.6
realy = 10.6
                
xbins = int(np.ceil(realx) / gridsize)
ybins = int(np.ceil(realy) / gridsize)


with open(agenttrace, 'r') as user_data:
    reader = csv.reader(user_data, delimiter=',', quotechar=',')
    for row in reader:
        if (reader.line_num > 1):
            oracle_raw.append([float(row[2]), float(row[3]), str(row[5])])

oracle = []
for i in range(len(oracle_raw)):
     oracle.append([oracle_raw[i][0] / gridsize, oracle_raw[i][1] / gridsize , oracle_raw[i][2]])
            
for d in oracle:
    gridized_data.append([int(np.floor(d[0])), int(np.floor(d[1]))])        

beacons = np.array([[1, 1], [3, 4.9], [1.35, 7.7], [3.3, 10.5], [3.3, 10.5], [6, 0.5], [0.5, 3.3], [6, 10.5]])    


for i in range(len(beacons)):
    beacons[i][0] = beacons[i][0]/gridsize
    beacons[i][1] = beacons[i][1]/gridsize

TM = transition_matrix2(gridized_data, xbins, ybins)

In [ ]:
TM

In [ ]:
def CumulativeSum(m):
    return m.cumsum()

def returnobj(cdf):
    # print("aaaaaaaaaaaabbbbbbb:")
    # print(cdf)
    randnum = np.random.random()
    for i in range(len(cdf) - 1):
        if((randnum>cdf[i]) and (randnum<=cdf[i + 1])):
            return i

def IndexToState(index, xbins, ybins):
    # print("aaaaaa:")
    # print(index)
    # x = np.floor(index/(ybins + 1))
    # y = index % (ybins + 1)
    y = int(np.floor(index / xbins))
    x = index - xbins * y
    return x, y

def StateToIndex(x, y, xbins, ybins):
    i = xbins * np.floor(y) + np.floor(x)
    # print("aaaa")
    # print(x)
    # print(y)
    # print(i)
    return int(i)
    
    # return int((ybins + 1) * np.floor(x) + np.floor(y))
        
def NextStates(TM, particles, xbins, ybins):
    cdf = np.apply_along_axis(CumulativeSum, 1, TM)
    zero = np.array([0]) # np.zeros((len(cdf[:,1]))
    
    c = np.tile(zero[np.newaxis,:], (cdf.shape[0],1))
    cdf = np.concatenate((c, cdf), axis=1)
    
    # cdf = np.concatenate([np.zeros((len(cdf[:,1])), 1), cdf])
    # next_states = []
    #print(len(particles))
    next_states = np.empty((len(particles), 2))
    
    for i in range(len(particles)):
        current_state_x = particles[i, 0]
        current_state_y = particles[i, 1]
        
        row = StateToIndex(current_state_x, current_state_y, xbins, ybins)
        # print(row)
        next_state_index = returnobj(cdf[row])
        next_states[i, 0] = IndexToState(next_state_index, xbins, ybins)[0]
        next_states[i, 1] = IndexToState(next_state_index, xbins, ybins)[1]
    
    # for (state, state_p) in zip(transitions,transitions[1:]):
        # i = state[0]*xbins + state[1]
        # j = state_p[0]*xbins + state_p[1]
    
    # print(len(next_states))
    # print(next_states[:,0])
    return next_states

In [ ]:
########################
##### Predict Step #####
########################

def predict(particles, xbins, ybins):
    next_states = NextStates(TM, particles, xbins, ybins)
    particles[:, 0] = next_states[:, 0]
    particles[:, 1] = next_states[:, 1]
    # print("MAX location is" + str(np.max(particles)))
    return particles

In [ ]:
TM

In [ ]:
from filterpy.monte_carlo import systematic_resample
from numpy.linalg import norm
from numpy.random import randn
import scipy.stats
import time

def DistanceToRSSI(agent_pos):
    rssi_values = []
    for b in beacons:
        dist = int(round(np.linalg.norm(b - agent_pos)))
        
        # Max reading is (7 / gridsize) meters
        if (dist > int(7 / gridsize)):
            dist = int(7 / gridsize)
        
        # print(dist)
        mean = beacons_mean[dist - 1]
        std = beacons_stds[dist - 1]
        
        sample = np.random.normal(mean, std, 1)
        rssi_values.append(sample[0])
        
    return rssi_values

def ActivitiesSensorGeneration(data):
    loc_x = data[0]
    loc_y = data[1]
    print(loc_x)
    print(loc_y)
    for motion in motion_sensors:
        print(motion.x)
        print(motion.y)
        print(motion.r)
        circ = Circle((motion.x, motion.y), motion.r)
        if (circ.contains_point([loc_x, loc_y])):
            print("hello")
            print(motion.x)
            print(motion.y)
            motion.Fire()
    
    for sensor in activity_sensors:
        activity = data[2].lower()
        if (activity == 'sleep' or activity == 'sit' or activity == 'eat'):
            if(type(sensor) == PressureSensor):
                # print("1")
                # print(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]))
                if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 2):
                    # print("fired!")
                    sensor.Fire()

        elif (activity == 'wash' or activity == 'clean'):
            if(type(sensor) == WaterFlowSensor):
                # print("2")
                # print(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]))
                if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 2):
                    sensor.Fire()

        elif (activity == 'exercise' or activity == 'cook'):
            if(type(sensor) == Accelerometer):
                if(sensor.attached_object == 'Agent'):
                    sensor.Fire()
                    
                elif(sensor.attached_object == 'Refrigerator-Side-by-Side-with_Ice_and_Water_Dispenser'):
                    # print("3")
                    # print(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]))
                    if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 2 / gridsize):
                        sensor.Fire()
                    
        elif (activity == 'entertainment'):
            if(type(sensor) == ElectricitySensor):
                if(sensor.attached_object == 'TV - Flat Screen'):
                    sensor.Fire()

def ResetSensorStates():
    for sensor in activity_sensors:
        sensor.Deactive()
        
    for motion in motion_sensors:
        motion.Deactive()
        
def RecognizeActivity(current_location, previous_location, particles):
    for motion in motion_sensors:
        if (motion.active):
            particles[:,0] = motion.x
            particles[:,1] = motion.y
    
    for sensor in activity_sensors:
        if (sensor.active):
            if (type(sensor) == Accelerometer):
                if (sensor.attached_object == 'Refrigerator-Side-by-Side-with_Ice_and_Water_Dispenser'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'cook'
            
            elif (type(sensor) == ElectricitySensor):
                if (sensor.attached_object == 'Oven Tower Unit' or sensor.attached_object == 'Counter Top w Sink' 
                    or sensor.attached_object == 'Counter Top - Corner'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'cook'
                
                if (sensor.attached_object == 'TV - Flat Screen'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'entertainment'
                
            elif (type(sensor) == WaterFlowSensor):
                if (sensor.attached_object == 'Sink Kitchen-Single'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'clean'
                
            elif (type(sensor) == PressureSensor):
                if (sensor.attached_object == 'Chair-Breuer' or sensor.attached_object == 'Dining Table w Chairs'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'sit'
                
                elif (sensor.attached_object == 'Chair-Corbu' or sensor.attached_object == 'Sofa-Corbu'
                      or sensor.attached_object == 'bed'):
                    particles[:,0] = sensor.x
                    particles[:,1] = sensor.y
                    return 'sleep'
                
    if (distance.euclidean(current_location, previous_location) <= 0.05):
        return 'nothing'
        
    else:
        return 'walk'

    

        
def run_pf1(N, iters=18, sensor_std_err=.1, do_plot=True, plot_particles=False, xlim=(0, xbins), ylim=(0, ybins), initial_x=None):
    # plt.figure()
    # create particles and weights
    fig, ax = plt.subplots(1,1)
    particles = create_uniform_particles((0,xbins), (0,ybins), (0, 1.28), N)    
    weights = np.ones(N) / N
    xs = []
    step = 1
    weights = np.ones(N) / N
    for gt in range(len(oracle)):
        step = step + 1
        agent_trace = oracle[gt]
        agent_pos = [agent_trace[0], agent_trace[1]]
        
        ActivitiesSensorGeneration(agent_trace)
        
        particles = predict(particles, xbins, ybins)        
        
        try:
            activity = RecognizeActivity(xs[len(xs) - 1], xs[len(xs) - 2], particles)
        except:
            activity = RecognizeActivity(0, 0, particles)
        
        zs =  DistanceToRSSI(agent_pos)
        
        
        
        update(particles, weights, z = zs, R = sensor_std_err)
        
        # resample if too few effective particles
        
        
        # if neff(weights) < N/2:
        # indexes = systematic_resample(weights)
        # simple_resample(particles, weights)
        # assert np.allclose(weights, 1/N)
          
            
        
        mu, var = estimate(particles, weights)
        xs.append(mu)
        
        # print(agent_trace[2] + ' VS ' + RecognizeActivity(xs[len(xs) - 1], xs[len(xs) - 2]))
        
        # if plot_particles:
            # plt.scatter(particles[:, 0], particles[:, 1], color='k', marker=',', s=1)
        
        # p1 = plt.scatter(agent_pos[0], agent_pos[1], marker='+', color='k', s=180, lw=3)
        # p2 = plt.scatter(mu[0], mu[1], marker='s', color='r')
        p1 = ax.scatter(agent_pos[0], agent_pos[1], marker='+', color='k', s=180, lw=3)
        p2 = ax.scatter(mu[0], mu[1], marker='s', color='r')
    
        xx = []
        yy = []
        for mo in motion_sensors:
            xx.append(mo.x)
            yy.append(mo.y)
            
        if plot_particles:
            alpha = .2
            if N > 5000:
                alpha *= np.sqrt(5000)/np.sqrt(N)           
            # plt.scatter(particles[:, 0], particles[:, 1], alpha=alpha, color='b')
            # plt.scatter(xx, yy, alpha=alpha, color='g')
        
        xs_new = np.array(xs)
        # plt.plot(xs[:, 0], xs[:, 1])
        plt.legend([p1, p2], ['Synthetic Ground Truth', 'Particle Filter Estimation'], loc=1, numpoints=1)
        plt.xlim(*xlim)
        plt.ylim(*ylim)
        # plt.xlabel = "Synthetic Ground Truth Activity: " str(agent_trace[2]) + " Estimated Activity: " + str(RecognizeActivity(xs[len(xs) - 1], xs[len(xs) - 2]))
        fig.suptitle("Synthetic Ground Truth Activity: " + 
                     str(agent_trace[2]) + 
                     " |  Estimated Activity: " + 
                     str(activity))
        # print('final position error, variance:\n\t', mu - np.array([iters, iters]), var)
        # plt.show()
        fig.canvas.draw()
        time.sleep(0.1)
        # plt.clf()
        ax.cla()
        ResetSensorStates()
        

In [ ]:
%matplotlib notebook
from numpy.random import seed
seed(2) 
run_pf1(N=1000, plot_particles=True)

In [ ]:
Circle(int(3), int(4), int(5))

In [ ]:
motion_sensors

In [ ]:
xbins * ybins

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Our 2-dimensional distribution will be over variables X and Y
N = 100
X = np.linspace(49, 51, N)
Y = np.linspace(49, 51, N)
X, Y = np.meshgrid(X, Y)

# Mean vector and covariance matrix
mu = np.array([50., 50.])
Sigma = np.array([[ 0.2 , 0.], [0.,  0.2]])

# Pack X and Y into a single 3-dimensional array
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos.

    pos is an array constructed by packing the meshed arrays of variables
    x_1, x_2, x_3, ..., x_k into its _last_ dimension.

    """

    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

# The distribution on the variables X, Y packed into pos.
Z = multivariate_gaussian(pos, mu, Sigma)
grid = Z
# Create a surface plot and projected filled contour plot under it.
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, rstride=3, cstride=3, linewidth=1, antialiased=True, cmap=cm.viridis)

# cset = ax.contourf(X, Y, Z, zdir='z', offset=-0.15, cmap=cm.viridis)

# Adjust the limits, ticks and view angle
# ax.set_zlim(-0.15,0.2)
# ax.set_zticks(np.linspace(0,0.2,5))
ax.view_init(30, 50)

plt.show()

import pylab as pl
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm

# im = plt.matshow(zip(*error_heatmap_averaged), cmap=pl.cm.RdYlGn_r)
im = plt.matshow(grid, cmap=pl.cm.OrRd)

plt.colorbar(im)
plt.show()

In [ ]:
import pylab as pl
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm

# im = plt.matshow(zip(*error_heatmap_averaged), cmap=pl.cm.RdYlGn_r)
im = plt.matshow(grid, cmap=pl.cm.OrRd)

plt.colorbar(im)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as sp

x = np.random.randn(100000)
y = np.random.randn(100000)


X = sp.filters.gaussian_filter(x, sigma = 2, order = 0)
Y = sp.filters.gaussian_filter(y, sigma = 2, order = 0)

# normal distribution center at x=0 and y=5
fig1 = plt.subplot(2,2,1)
plt.hist2d(x, y, bins=40)
plt.colorbar()
plt.title('Heatmap without smoothing')
plt.xlabel("X")
plt.ylabel("Y")

In [ ]:
# Add a colorbar if necessary
plt.hexbin(x, y, gridsize=(25,25), cmap=plt.cm.Purples_r)

plt.show()
